#### Lab1：Regression

In [10]:
import pandas as pd
# load train_set.csv
train_set = pd.read_csv('./dataset/train_set.csv')
# load test_set.csv
test_set = pd.read_csv('./dataset/test_set.csv')

# check the shape of train_set & test_set
print("Shape of train_set:", train_set.shape, "; Shape of test_set:", test_set.shape)
# describe the train set
train_set.describe()



Shape of train_set: (15419, 13) ; Shape of test_set: (3949, 13)


,median_income,house_age,avg_rooms,avg_bedrooms,block_population,avg_occupancy,latitude,longitude,house_value,diag_coord,bed_per_rooms,rooms_per_occup,pop_per_occup
count,15419.000000,15419.000000,15419.000000,15419.000000,15419.00000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000
mean,3.674941,28.528115,5.207766,1.063198,1427.27972,2.937544,35.642385,-119.571472,1.923097,-83.929087,0.214705,1.871840,498.665867
std,1.546898,12.452558,1.168424,0.112638,1012.73317,0.731269,2.144430,1.999935,0.969086,0.794560,0.055790,0.562230,355.962390
min,0.499900,1.000000,1.130435,0.333333,8.00000,1.161290,32.540000,-124.350000,0.149990,-85.870000,0.113535,0.311943,2.000000
25%,2.536400,18.000000,4.408248,1.004929,804.00000,2.451423,33.930000,-121.770000,1.168500,-84.410000,0.177556,1.493691,284.000000
50%,3.455100,29.000000,5.170543,1.047256,1184.00000,2.842491,34.270000,-118.510000,1.739000,-84.170000,0.204441,1.906606,412.000000
75%,4.583050,37.000000,5.942079,1.095962,1743.50000,3.304499,37.730000,-118.000000,2.476000,-83.510000,0.240944,2.235102,606.000000
max,10.989100,52.000000,14.360000,1.983784,8997.00000,6.497585,41.950000,-114.550000,4.991000,-78.290000,0.922414,6.033613,4204.000000


In [11]:
# split input and output of both train_set and test_set
train_x = train_set.drop(['house_value'], axis=1)
train_y = train_set['house_value']
test_x = test_set.drop(['house_value'], axis=1)
test_y = test_set['house_value']
# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# transform the sets into numpy array
#train_x = train_x.values
train_y = train_y.values
#test_x = test_x.values
test_y = test_y.values

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# import sklearn's linear, lasso, ridge, SVM, k-neighbors, descision tree, GBDT regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

# import sklearn's mean squared error and r2 score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import time

def train_model(model, train_x, train_y):
    # start timing
    start_time = time.time()
    # train the model
    model.fit(train_x, train_y)
    # end timing
    end_time = time.time()
    training_time = end_time - start_time
    # calculate the MSE, r2 & adjested r2 score of training set
    pred_y = model.predict(train_x)
    mse = mean_squared_error(train_y, pred_y)
    r2 = r2_score(train_y, pred_y)
    adj_r2 = 1 - (1 - r2) * (len(train_y) - 1) / (len(train_y) - train_x.shape[1] - 1)
    return training_time, mse, r2, adj_r2
    

def test_model(model, test_x, test_y):
    # predict the output of test set
    pred_y = model.predict(test_x)
    # calculate the mean squared error
    mse = mean_squared_error(test_y, pred_y)
    # calculate the r2 score
    r2 = r2_score(test_y, pred_y)
    # calcultae the adjusted r2 score
    adj_r2 = 1 - (1 - r2) * (len(test_y) - 1) / (len(test_y) - test_x.shape[1])
    return mse, r2, adj_r2


# create a function to train and test the models
def train_test_model(model, train_x, train_y, test_x, test_y):
    # train the model
    train_result = train_model(model, train_x, train_y)
    # test the model
    test_result = test_model(model, test_x, test_y)
    # concat train & test result
    result = train_result + test_result
    return result



In [ ]:
# create a dict with all the models
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(random_state=42),
    'Ridge': Ridge(random_state=42),
    'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(random_state=42),
    'GradientBoostingRegressor': GradientBoostingRegressor(random_state=42)
}

# create a pandas dataframe to log the results
result_df = pd.DataFrame(columns=['model_name', 'training_time', 'train_mse', 'train_r2', 'train_adj_r2', 'test_mse', 'test_r2', 'test_adj_r2'])

# train and test all the models
for name, model in models.items():
    result = train_test_model(model, train_x, train_y, test_x, test_y)
    # append the name and result to result_df
    result_df = result_df.append({'model_name': name, 'training_time': result[0], 'train_mse': result[1], 'train_r2': result[2], 'train_adj_r2': result[3], 'test_mse': result[4], 'test_r2': result[5], 'test_adj_r2': result[6]}, ignore_index=True)

In [ ]:
from IPython.display import display
# display the result_df
display(result_df)
result_df.to_csv('./result/regressionPref.csv', index=False, float_format='%.4f')

,model_name,training_time,train_mse,train_r2,train_adj_r2,test_mse,test_r2,test_adj_r2
0,LinearRegression,0.017999,0.327036,0.651743,0.651472,5.198885e+21,-5.452132e+21,-5.467365e+21
1,Lasso,0.006996,0.939066,0.000000,-0.000779,9.535910e-01,-4.205992e-05,-2.836183e-03
2,Ridge,0.008000,0.327102,0.651673,0.651402,3.536866e-01,6.290847e-01,6.280484e-01
3,SVR,8.920308,0.211019,0.775289,0.775114,2.576389e-01,7.298111e-01,7.290561e-01
4,KNeighborsRegressor,0.021000,0.174289,0.814402,0.814257,2.901947e-01,6.956694e-01,6.948191e-01
5,DecisionTreeRegressor,0.161167,0.000000,1.000000,1.000000,3.514345e-01,6.314466e-01,6.304168e-01
6,GradientBoostingRegressor,3.414722,0.190695,0.796931,0.796773,2.329652e-01,7.556867e-01,7.550041e-01
